
- make sure that the GSA is attached to your cluster `fe-prod-general-sa@fe-prod-dbx.iam.gserviceaccount.com`
- GSA has `Secrets Manager Admin` role assigned
- follow GCP [docs](https://cloud.google.com/secret-manager/docs/overview)

In [0]:
%pip install google-cloud-secret-manager
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached google_cloud_secret_manager-2.19.0-py2.py3-none-any.whl (178 kB)
  Using cached grpc_google_iam_v1-0.13.0-py2.py3-none-any.whl (25 kB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
def list_secrets(project_id: str) -> None:
    """
    List all secrets in the given project.
    """
    # Import the Secret Manager client library.
    from google.cloud import secretmanager
    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()
    # Build the resource name of the parent project.
    parent = f"projects/{project_id}"
    # List all secrets.
    for secret in client.list_secrets(request={"parent": parent}):
        print(f"Found secret: {secret.name}")

In [0]:
list_secrets("fe-prod-dbx")

Found secret: projects/567024742550/secrets/bk-secret1


In [0]:
from google.cloud import secretmanager

def get_secret(project_id: str, secret_id: str) -> secretmanager.GetSecretRequest:
    """
    Get information about the given secret. This only returns metadata about
    the secret container, not any secret material.
    """

    # Import the Secret Manager client library.
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret.
    name = client.secret_path(project_id, secret_id)

    # Get the secret.
    response = client.get_secret(request={"name": name})

    # Get the replication policy.
    if "automatic" in response.replication:
        replication = "AUTOMATIC"
    elif "user_managed" in response.replication:
        replication = "MANAGED"
    else:
        raise Exception(f"Unknown replication {response.replication}")

    # Print data about the secret.
    print(f"Got secret {response.name} with replication policy {replication}")

In [0]:
# Import the Secret Manager client library.
from google.cloud import secretmanager

# GCP project in which to store secrets in Secret Manager.
project_id = "fe-prod-dbx"
# ID of the secret to create.
secret_id = "fe-prod-demo-secret1"
# Create the Secret Manager client.
client = secretmanager.SecretManagerServiceClient()
# Build the parent name from the project.
parent = f"projects/{project_id}"
# Create the parent secret.
secret = client.create_secret(
    request={
        "parent": parent,
        "secret_id": secret_id,
        "secret": {"replication": {"automatic": {}}},
    }
)
# Add the secret version.
version = client.add_secret_version(
    request={"parent": secret.name, "payload": {"data": b"hello world!"}}
)
# Access the secret version.
response = client.access_secret_version(request={"name": version.name})
# Print the secret payload.
#
# WARNING: Do not print the secret in a production environment - this
# snippet is showing how to access the secret material.
payload = response.payload.data.decode("UTF-8")
print(f"Plaintext: {payload}")


Plaintext: hello world!


In [0]:
{secret:"valu", expire:""}